In [1]:
!pip install pymysql sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.0 MB/s eta 0:00:00


In [2]:
sql_file_path = '/content/project/sql/sql.sql'

In [3]:
# with open(sql_file_path, 'r', encoding='utf-8') as f:
#     sql_query = f.read()

# print(sql_query[:500])

In [4]:
import sqlite3
import pandas as pd

# sqlite 메모리 DB 생성
conn = sqlite3.connect(':memory:')

# sql 파일 읽기
with open('/content/project/sql/sql.sql', 'r', encoding='utf-8') as f:
    sql_script = f.read()

# sql 스크립트 실행 (테이블 생성 + 데이터 삽입)
conn.executescript(sql_script)

# 쿼리 실행 후 데이터프레임으로 변환
df = pd.read_sql_query('SELECT * FROM ev', conn)

print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/project/sql/sql.sql'

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# CSV 파일 경로
csv_path = '/content/project/csv/registration status.csv'  # 실제 파일 경로로 변경하세요

# CSV 불러오기, 인코딩이 UTF-8이 아닐 수도 있으니 인코딩 옵션 확인 필요
df = pd.read_csv(csv_path, encoding='utf-8')

# 데이터 확인
print(df.head())


In [ ]:
from folium.plugins import HeatMap

# heat_data 생성
heat_data = []
for region in region_coords.keys():
    value = int(row[region])  # numpy.int64 -> int 변환
    lat, lon = region_coords[region]
    heat_data.append([lat, lon, value])

# 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=7)

# 히트맵 추가 (max_val 제거)
HeatMap(
    heat_data,
    radius=45,       # 반경 키우기
    blur=25,         # 흐림 정도 증가
    min_opacity=0.4  # 최소 불투명도
).add_to(m)


m


In [ ]:
print(map_df['값'].isna().sum())
print(map_df.dtypes)


In [ ]:
map_df['값'] = pd.to_numeric(map_df['값'], errors='coerce')
map_df = map_df.dropna(subset=['값'])

In [ ]:
for f in geo_data['features']:
    print(f['properties']['name'])  # 이 값과 map_df['지역명'] 이 일치해야 함

In [ ]:
# 1. 매핑
map_df['지역명'] = map_df['지역명'].map(지역명_매핑)

# 2. 숫자화
map_df['값'] = pd.to_numeric(map_df['값'], errors='coerce')
map_df = map_df.dropna(subset=['값'])

# 3. Choropleth 다시 실행


In [ ]:
import pandas as pd
import folium
import json

# 1. CSV 로드
df = pd.read_csv('/content/project/csv/registration status.csv')

# 2. 최신 월 데이터만 추출
latest_row = df.iloc[-1]  # 가장 마지막 행 (최신 데이터)
지역별_값 = latest_row.drop(labels=['년월', '합계'])  # '년월'과 '합계' 제거

# 3. long 포맷으로 변환
map_df = 지역별_값.reset_index()
map_df.columns = ['지역명', '값']
map_df['값'] = pd.to_numeric(map_df['값'], errors='coerce')

# 4. GeoJSON 불러오기
with open('/content/project/map/skorea_provinces_geo.json', encoding='utf-8') as f:
    geo_data = json.load(f)

# 5. 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=7)

# 6. Choropleth 시각화
folium.Choropleth(
    geo_data=geo_data,
    data=map_df,
    columns=['지역명', '값'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='지역별 수치',
    nan_fill_color='lightgrey'
).add_to(m)

# 7. 출력
m


In [ ]:
import pandas as pd

# 예시: 엑셀 또는 CSV에서 바로 읽어온 형태라고 가정
data = {
    '서울': [88212],
    '부산': [48623],
    '대구': [37027],
    '인천': [62986],
    '광주': [16641],
    '대전': [22843],
    '울산': [10786],
    '세종': [5934],
    '경기': [169781],
    '강원': [22455],
    '충북': [28860],
    '충남': [34545],
    '전북': [26902],
    '전남': [35169],
    '경북': [37957],
    '경남': [52856],
    '제주': [52529]
}

df = pd.DataFrame(data)

# 행 1개짜리 wide 형태 → long 형태로 변환
map_df = df.melt(var_name='지역명', value_name='값')
print(map_df)


In [ ]:
지역명_매핑 = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '울산': '울산광역시',
    '세종': '세종특별자치시',
    '경기': '경기도',
    '강원': '강원도',
    '충북': '충청북도',
    '충남': '충청남도',
    '전북': '전라북도',
    '전남': '전라남도',
    '경북': '경상북도',
    '경남': '경상남도',
    '제주': '제주특별자치도'
}

map_df['지역명'] = map_df['지역명'].map(지역명_매핑)


In [ ]:
%%writefile app.py

import folium
import json

# GeoJSON 파일 불러오기
with open('/content/project/map/skorea_provinces_geo.json', encoding='utf-8') as f:
    geo_data = json.load(f)

# 데이터 예시 (map_df)
# map_df에는 '지역명'과 '값' 컬럼이 있어야 합니다.

# '지역명'을 key로 '값'을 value로 하는 딕셔너리 생성
value_dict = map_df.set_index('지역명')['값'].to_dict()

# geojson 각 feature에 'value' 속성 추가
for feature in geo_data['features']:
    region_name = feature['properties']['name']
    feature['properties']['value'] = value_dict.get(region_name, '값 없음')

# 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=7)

# Choropleth 추가
folium.Choropleth(
    geo_data=geo_data,
    data=map_df,
    columns=['지역명', '값'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='지역별 값'
).add_to(m)

# GeoJson 레이어에 툴팁 추가 (지역명과 값 같이 표시)
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'transparent',
        'weight': 0,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'value'],
        aliases=['', ''],
        localize=True,
        style="""
        font-size: 16px;
        font-weight: bold;
        background-color: white;
        border: 1px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """
    )

).add_to(m)

m

In [ ]:
# import json

# with open('/content/project/map/map.geojson', encoding='utf-8') as f:
#     geo_data = json.load(f)

# # 첫 번째 피처의 속성 확인
# print(geo_data['features'][0]['properties'])


In [ ]:
import pandas as pd
import folium
import json

# 데이터 숫자형으로 변환 및 NaN 처리
df['값'] = pd.to_numeric(df['값'], errors='coerce')
df = df.dropna(subset=['값'])

# Choropleth 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=7)

folium.Choropleth(
    geo_data=geo_data,
    data=df,
    columns=['지역명', '값'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='데이터 값'
).add_to(m)

m


In [ ]:
print(df.dtypes)
print(df['값'].apply(type).value_counts())

In [ ]:
df = df.dropna(subset=['값']).copy()

In [ ]:
print(df[df['값'].isna()])


In [ ]:
print(df['값'].head(10))
print(df['지역명'].head(10))


In [ ]:
folium.Choropleth(
    geo_data=geo_data,
    data=df,
    columns=['지역명', '값'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='데이터 값'
).add_to(m)


In [ ]:
for i, v in enumerate(df['값']):
    if not isinstance(v, (int, float)):
        print(i, v, type(v))

In [ ]:
print(df.head())               # 데이터프레임 일부 출력
print(df.dtypes)               # 컬럼별 타입 출력

print(df['값'].apply(type).value_counts())  # '값' 컬럼에 어떤 타입들이 있는지

# 숫자 변환 불가능한 값 출력
for i, v in enumerate(df['값']):
    if not (isinstance(v, (int, float)) and not isinstance(v, bool)):
        print(f"Non-numeric at index {i}: value={v}, type={type(v)}")


In [ ]:
print(df['값'].dtype)
print(df['값'].head())

In [ ]:
df['값'] = pd.to_numeric(df['값'], errors='coerce')

In [ ]:
df = df.dropna(subset=['값'])
# 또는
df['값'] = df['값'].fillna(0)


In [ ]:
print(df.columns.tolist())

In [ ]:
import os

# 드라이브 내 파일 구조 확인
path = '/content/drive/MyDrive'
for root, dirs, files in os.walk(path):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
import streamlit as st

# 가상의 데이터베이스 (실제 사용 시 이 부분을 데이터베이스 연결로 대체)
database = [
    "사과는 빨간색 과일입니다.",
    "바나나는 노란색 과일입니다.",
    "오렌지는 주황색 과일입니다.",
    "포도는 보라색 또는 녹색 과일입니다.",
    "키위는 갈색 껍질에 녹색 속을 가진 과일입니다."
]

# 검색 함수
def search(query):
    return [item for item in database if query.lower() in item.lower()]

# 제목
st.title("과일 검색 앱")

# 검색어 입력창
query = st.text_input("검색어를 입력하세요:")

# 검색 버튼
if st.button("검색"):
    if query:
        results = search(query)
        if results:
            st.subheader("검색 결과:")
            for result in results:
                st.write(result)
        else:
            st.write("검색 결과가 없습니다.")
    else:
        st.write("검색어를 입력해주세요.")

In [ ]:
!pip install streamlit

In [ ]:
%%writefile my_app.py
import streamlit as st

st.title("간단한 스트림릿 테스트")
st.write("이건 Streamlit 앱입니다.")


In [ ]:
!pip install streamlit pyngrok --quiet

In [ ]:
%%writefile app.py
import streamlit as st

database = [
    "사과는 빨간색 과일입니다.",
    "바나나는 노란색 과일입니다.",
    "오렌지는 주황색 과일입니다.",
    "포도는 보라색 또는 녹색 과일입니다.",
    "키위는 갈색 껍질에 녹색 속을 가진 과일입니다."
]

def search(query):
    return [item for item in database if query.lower() in item.lower()]

st.title("과일 검색 앱")
query = st.text_input("검색어를 입력하세요:")

if st.button("검색"):
    if query:
        results = search(query)
        if results:
            st.subheader("검색 결과:")
            for result in results:
                st.write(result)
        else:
            st.write("검색 결과가 없습니다.")
    else:
        st.write("검색어를 입력해주세요.")

In [ ]:
import streamlit as st

# 가상의 데이터베이스 (실제 사용 시 이 부분을 데이터베이스 연결로 대체)
database = [
    "사과는 빨간색 과일입니다.",
    "바나나는 노란색 과일입니다.",
    "오렌지는 주황색 과일입니다.",
    "포도는 보라색 또는 녹색 과일입니다.",
    "키위는 갈색 껍질에 녹색 속을 가진 과일입니다."
]

# 검색 함수
def search(query):
    return [item for item in database if query.lower() in item.lower()]

# 제목
st.title("과일 검색 앱")

# 검색어 입력창
query = st.text_input("검색어를 입력하세요:")

# 검색 버튼
if st.button("검색"):
    if query:
        results = search(query)
        if results:
            st.subheader("검색 결과:")
            for result in results:
                st.write(result)
        else:
            st.write("검색 결과가 없습니다.")
    else:
        st.write("검색어를 입력해주세요.")